### **WebApp을 만들어보자**

Streamlit 는 머신러닝 결과를 손쉽게 배포할 수 있는 패키지입니다. 일반적으로 웹앱은 HTML, CSS, JavaScript 등의 기술이 있어야 원하는 웹앱을 만들 수 있습니다. 파이썬만으로 웹앱을 만들 수 있게 해 주는 패키지가 Streamlit 입니다. Steamlit 을 써보고 "바로 이거야" 하고 감탄했던 기억이 납니다. 이런 상황을 생각해 보세요. 오늘 갑자기 지방출장이 생겼습니다. 회사 노트북으로는 파이썬을 돌리지 못하니 어떤 종목을 추천받아 매수를 해야 할지 궁금합니다. 만약 웹앱을 만들어 인터넷으로 추천 종목 결과물을 받아보면 얼마나 좋을까요? streamlit 을 이용하면 아주 쉽게 할 수 있습니다. 더 나아가 매수/매도 결과 등도 웹앱을 이용해 조회 할 수 있습니다.    

이 책에서 구현하고자 하는 웹앱은 장 마감 후, 당일 날짜를 'YYYY-MM-DD' 형식으로 입력하면 내일 매수 추천 종목이 뜨는 WebApp입니다. 먼저 만들어 놓은 종목 추천 함수를 테스트 해 봅니다. 추천함수에 필요한 데이터는 종목별로 Loop 를 돌리기 위한 'kosdaq_list.pkl' 파일과 모델 데이터 "gam.pkl" 입니다. 이 두 파일이 같은 폴더에 있어야 됩니다. 

In [10]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import datetime
import pickle
import time

In [8]:
def select_stocks(today_dt):
    
    today = datetime.datetime.strptime(today_dt, '%Y-%m-%d')
    start_dt = today - datetime.timedelta(days=100) # 100 일전 데이터 부터 시작 - 피쳐 엔지니어링은 최소 60 개의 일봉이 필요함
    print(start_dt, today_dt)

    kosdaq_list = pd.read_pickle('kosdaq_list.pkl')

    price_data = pd.DataFrame()

    for code, name in zip(kosdaq_list['code'], kosdaq_list['name']):  # 코스닥 모든 종목에서 대하여 반복
        daily_price = fdr.DataReader(code, start = start_dt, end = today_dt) # 종목, 일봉, 데이터 갯수

        daily_price['code'] = code
        daily_price['name'] = name
        price_data = pd.concat([price_data, daily_price], axis=0)   

    price_data.index.name = 'date'
    price_data.columns= price_data.columns.str.lower() # 컬럼 이름 소문자로 변경

    kosdaq_index = fdr.DataReader('KQ11', start = start_dt, end = today_dt) # 데이터 호출
    kosdaq_index.columns = ['close','open','high','low','volume','change'] # 컬럼명 변경
    kosdaq_index.index.name='date' # 인덱스 이름 생성
    kosdaq_index.sort_index(inplace=True) # 인덱스(날짜) 로 정렬 
    kosdaq_index['kosdaq_return'] = kosdaq_index['close']/kosdaq_index['close'].shift(1) # 수익율 : 전 날 종가대비 당일 종가

    merged = price_data.merge(kosdaq_index['kosdaq_return'], left_index=True, right_index=True, how='left')

    return_all = pd.DataFrame()

    for code in kosdaq_list['code']:  

        stock_return = merged[merged['code']==code].sort_index()
        stock_return['return'] = stock_return['close']/stock_return['close'].shift(1) # 종목별 전일 종가 대비 당일 종가 수익율
        c1 = (stock_return['kosdaq_return'] < 1) # 수익율 1 보다 작음. 당일 종가가 전일 종가보다 낮음 (코스닥 지표)
        c2 = (stock_return['return'] > 1) # 수익율 1 보다 큼. 당일 종가가 전일 종가보다 큼 (개별 종목)
        stock_return['win_market'] = np.where((c1&c2), 1, 0) # C1 과 C2 조건을 동시에 만족하면 1, 아니면 0
        return_all = pd.concat([return_all, stock_return], axis=0) 

    return_all.dropna(inplace=True)    

    model_inputs = pd.DataFrame()

    for code, name, sector in zip(kosdaq_list['code'], kosdaq_list['name'], kosdaq_list['sector']):

        data = return_all[return_all['code']==code].sort_index().copy()    

        # 가격변동성이 크고, 거래량이 몰린 종목이 주가가 상승한다
        data['price_mean'] = data['close'].rolling(20).mean()
        data['price_std'] = data['close'].rolling(20).std(ddof=0)
        data['price_z'] = (data['close'] - data['price_mean'])/data['price_std']    
        data['volume_mean'] = data['volume'].rolling(20).mean()
        data['volume_std'] = data['volume'].rolling(20).std(ddof=0)
        data['volume_z'] = (data['volume'] - data['volume_mean'])/data['volume_std']

        # 위꼬리가 긴 양봉이 자주발생한다.
        data['positive_candle'] = (data['close'] > data['open']).astype(int) # 양봉
        data['high/close'] = (data['positive_candle']==1)*(data['high']/data['close'] > 1.1).astype(int) # 양봉이면서 고가가 종가보다 높게 위치
        data['num_high/close'] =  data['high/close'].rolling(20).sum()
        data['long_candle'] = (data['positive_candle']==1)*(data['high']==data['close'])*\
        (data['low']==data['open'])*(data['close']/data['open'] > 1.2).astype(int) # 장대 양봉을 데이터로 표현
        data['num_long'] =  data['long_candle'].rolling(60).sum() # 지난 20 일 동안 장대양봉의 갯 수


         # 거래량이 종좀 터지며 매집의 흔적을 보인다   
        data['volume_mean'] = data['volume'].rolling(60).mean()
        data['volume_std'] = data['volume'].rolling(60).std()
        data['volume_z'] = (data['volume'] - data['volume_mean'])/data['volume_std'] # 거래량은 종목과 주가에 따라 다르기 떄문에 표준화한 값이 필요함
        data['z>1.96'] = (data['close'] > data['open'])*(data['volume_z'] > 1.65).astype(int) # 양봉이면서 거래량이 90%신뢰구간을 벗어난 날
        data['num_z>1.96'] =  data['z>1.96'].rolling(60).sum()  # 양봉이면서 거래량이 90% 신뢰구간을 벗어난 날을 카운트

        # 주가지수보다 더 좋은 수익율을 보여준다
        data['num_win_market'] = data['win_market'].rolling(60).sum() # 주가지수 수익율이 1 보다 작을 때, 종목 수익율이 1 보다 큰 날 수
        data['pct_win_market'] = (data['return']/data['kosdaq_return']).rolling(60).mean() # 주가지수 수익율 대비 종목 수익율


        # 동종업체 수익률보다 더 좋은 수익율을 보여준다.           
        data['return_mean'] = data['return'].rolling(60).mean() # 종목별 최근 60 일 수익율의 평균
        data['sector'] = sector    
        data['name'] = name

        data = data[(data['price_std']!=0) & (data['volume_std']!=0)]    

        model_inputs = pd.concat([data, model_inputs], axis=0)

    model_inputs['sector_return'] = model_inputs.groupby(['sector', model_inputs.index])['return'].transform(lambda x: x.mean()) # 섹터의 평균 수익율 계산
    model_inputs['return over sector'] = (model_inputs['return']/model_inputs['sector_return']) # 섹터 평균 수익률 대비 종목 수익률 계산
    model_inputs.dropna(inplace=True) # Missing 값 있는 행 모두 제거


    feature_list = ['price_z','volume_z','num_high/close','num_win_market','pct_win_market','return over sector']

    X = model_inputs.loc[today_dt][['code','name','return','kosdaq_return','close'] + feature_list].set_index('code') 

    with open("gam.pkl", "rb") as file:
        gam = pickle.load(file)     

    yhat = gam.predict_proba(X[feature_list])
    X['yhat'] = yhat

    tops = X[X['yhat'] >= 0.3].sort_values(by='yhat', ascending=False) # 스코어 0.3 이상 종목만 
    print(len(tops))    
     
    select_tops = tops[(tops['return'] > 1.03) & (tops['price_z'] < 0)][['name','return','price_z','yhat','return', 'kosdaq_return','close']]  # 기본 필터링 조건   
      
    if len(select_tops) > 1: # 최소한 2개 종목 - 추천 리스크 분산        
        return select_tops    
    
    else:
        return None

2022년 4월 1일을 테스트 한 결과 함수가 잘 작동하는 것을 확인할 수 있습니다.

In [9]:
select_tops = select_stocks('2022-04-01')
select_tops.style.set_table_attributes('style="font-size: 12px"').format(precision=3)

2021-12-22 00:00:00 2022-04-01
204


,name,return,price_z,yhat,return,kosdaq_return,close
code,,,,,,,
056090,에디슨INNO,1.080,-1.621,0.542,1.080,0.996,12800
024740,한일단조,1.062,-0.823,0.357,1.062,0.996,3185
083660,CSA 코스믹,1.035,-0.094,0.353,1.035,0.996,2080
122690,서진오토모티브,1.058,-0.114,0.321,1.058,0.996,3350
174880,장원테크,1.090,-0.366,0.302,1.090,0.996,1990


<br></br>
파이썬 파일 'stock_selection.py' 만들고 select_stocks() 함수를 복사해 넣습니다. 쥬피터랩을 쓰고 계신다면 File > New > Python File 에서 파일을 하나 만드시고 그 파일에 함수를 복사해서 넣으면 됩니다. stock_selection.py 파일을 동일한 폴더에 저장하신 후에, 아래와 같이 import 를 해 봅니다. 그리고 다시 함수를 호출해서 2022년 4월 1일 추천종목을 확인해 봅니다. 이상없이 함수가 호출되는 것을 확인했습니다.

In [12]:
import stock_selection
stock_selection.select_stocks('2022-04-01')

2021-12-22 00:00:00 2022-04-01
204


,name,return,price_z,yhat,return,kosdaq_return,close
code,,,,,,,
056090,에디슨INNO,1.080169,-1.620633,0.542471,1.080169,0.995807,12800
024740,한일단조,1.061667,-0.823414,0.356940,1.061667,0.995807,3185
083660,CSA 코스믹,1.034826,-0.094418,0.353097,1.034826,0.995807,2080
122690,서진오토모티브,1.058452,-0.113779,0.320923,1.058452,0.995807,3350
174880,장원테크,1.090411,-0.366128,0.301724,1.090411,0.995807,1990


<br></br>이제 main.py 파이썬 파일을 만들겠습니다. main.py 에서는 streamlit 을 import 할 것이므로 streamlit 을 설치합니다. (참고 링크 https://anaconda.org/conda-forge/streamlit)
윈도우즈를 쓰고 계신다면 아나콘다 Prompt 을 열고 pip install streamlit 로 설치가 가능합니다. 아래 그림과 적으신 후 'Enter' 를 누르시면 자동으로 설치가 됩니다. 가상환경을 별도로 만들지 않고 (base) 에 설치하도록 하겠습니다. 

<img src="Install_Streamlit.PNG" width="800" height="150"></img>

이제 Streamlit 이 Import 되는 지 확인해보겠습니다.

In [16]:
import streamlit

ImportError: DLL load failed while importing lib: 지정된 프로시저를 찾을 수 없습니다.